In [70]:
#180602_Kaggle_Word2Vec_Tutorial Part2

import pandas as pd

# Read data from files
train = pd.read_csv("/Users/haewonyum/.kaggle/competitions/word2vec-nlp-tutorial/labeledTrainData.tsv", header=0,\
                    delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("/Users/haewonyum/.kaggle/competitions/word2vec-nlp-tutorial/unlabeledTrainData.tsv", header=0,\
                    delimiter="\t", quoting=3)
test = pd.read_csv("/Users/haewonyum/.kaggle/competitions/word2vec-nlp-tutorial/testData.tsv", header=0,
                  delimiter="\t",quoting=3)

In [ ]:
# Verify the number of reviews that were read (100,000 in total)
# print(train.shape)
# print(unlabeled_train.shape)
# print(test.shape)
# print(train.columns)

In [14]:
## Import various modules for string cleaning
from bs4 import BeautifulSoup #to remove html
import numpy as np
import re #to use regular expression
from nltk.corpus import stopwords #to load predefined stopwords(불용어)

## Preprocessiong of raw text
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Remove non-letters
    review_text = re.sub("[^A-Za-z]"," ",review_text)

    # 3. Convert words to lower case and split them into individual words
    review_words = review_text.lower().split()
    
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        #In Python, searching a set is much faster than searching a list, so convert the stop words to a set
        stops = set(stopwords.words("english")) 
        review_words = [w for w in review_words if not w in stops]        
    # 5. Return a list of words
    return (review_words)

In [16]:
## check result
# train['review'][0]
# review_to_wordlist(train['review'][0])

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.
It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [28]:
import nltk.data

## Load the punkt tokenizer (sentence tokenizer)
# https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.punkt
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

## Define a function to split a review into parsed sentences
def review_to_sentence(review, tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences=[]
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_to_wordlist(raw_sentence))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [20]:
# test review_sentence function
print(train['review'][0])

"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."


In [23]:
# test review_sentence function - cont'd
# review_sentence(train['review'][0],tokenizer)

In [30]:
# Apply to train data
sentences = [] 
count =0
print("Parsing sentences from training set...")
for review in train['review']:
    count += 1
    sentences += review_to_sentence(review,tokenizer)
    if(count%5000==0):
        print("%d th reviews was appended"%(count))

print("Parsing sentences from unlabeled set")
for review in unlabeled_train['review']:
    count += 1
    sentences += review_to_sentence(review,tokenizer)
    if(count%5000==0):
        print("%d th reviews was appended"%(count))

Parsing sentences from training set


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/haewonyum/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.

10000 th sentences was appended
20000 th sentences was appended
Parsing sentences from unlabeled set


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


30000 th sentences was appended
40000 th sentences was appended


...

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


60000 th sentences was appended


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


70000 th sentences was appended


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [31]:
print(len(sentences))

795538


In [38]:
print (sentences[3])

['some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 'michael', 'jackson', 'so', 'unless', 'you', 'remotely', 'like', 'mj', 'in', 'anyway', 'then', 'you', 'are', 'going', 'to', 'hate', 'this', 'and', 'find', 'it', 'boring']


In [29]:
## Build Word2Vec Model
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages

import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%message)s', level=logging.INFO)

# set values for various parameters
num_features = 300 # Word vector dimensionality
min_word_count = 40 # minimum word count
num_workers = 4 #Number of threads to run in parallel
context = 10 # COntext window size
downsampling = 1e-3 # Downsample setting for frequent words

In [41]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, 
                          min_count=min_word_count, window=context, sample=downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...


--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2698, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-eba0d58afa17>", line 5, in <module>
    min_count=min_word_count, window=context, sample=downsampling)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/word2vec.py", line 527, in __init__
    fast_version=FAST_VERSION)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/base_any2vec.py", line 335, in __init__
 

  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2698, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(co

  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-eba0d58afa17>", line 5, in <module>
    min_count=min_word_count, window=context, sample=downsampling)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/word2vec.py", line 527, in __init__
    fast_version=FAST_VERSION)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/base_any2vec.py", line 335, in __init__
    self.build_vocab(sentences, trim_rule=trim_rule)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4.0-py3.6-macosx-10.7-x86_64.egg/gensim/models/base_any2vec.py", line 480, in build_vocab
    sentences, progress_per=progress_per, trim_rule=trim_rule)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/gensim-3.4

...

--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 440, in _handle_events
    self._handle_recv()
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 472, in _handle_recv
    self._run_callback(callback, msg)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/Users/haewonyum/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return

In [42]:
# Exploring the Model Results
# The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:
model.doesnt_match("man woman child kitchen".split())

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


'kitchen'

In [43]:
model.doesnt_match("france england germany berlin".split())

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [44]:
# ... although with the relatively small training set we used, it's certainly not perfect:
model.doesnt_match("paris berlin london austria".split())

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  


'paris'

In [45]:
# We can also use the "most_similar" function to get insight into the model's word clusters:
model.most_similar("man")

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('woman', 0.613494873046875),
 ('lady', 0.5942127704620361),
 ('lad', 0.5648837685585022),
 ('monk', 0.5243738889694214),
 ('men', 0.5216628909111023),
 ('millionaire', 0.5157448649406433),
 ('soldier', 0.512262225151062),
 ('guy', 0.5086937546730042),
 ('businessman', 0.5072619318962097),
 ('farmer', 0.506844699382782)]

In [47]:
model.most_similar("queen")

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bride', 0.6303467154502869),
 ('princess', 0.6170536279678345),
 ('victoria', 0.6126810908317566),
 ('maid', 0.5940853953361511),
 ('latifah', 0.587699830532074),
 ('mistress', 0.5831695199012756),
 ('belle', 0.5816861987113953),
 ('stepmother', 0.5795029997825623),
 ('prince', 0.5776283740997314),
 ('eva', 0.5677446126937866)]

In [49]:
model.most_similar("awful")

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7775144577026367),
 ('horrible', 0.744812548160553),
 ('atrocious', 0.7379262447357178),
 ('abysmal', 0.7241886854171753),
 ('dreadful', 0.7229462265968323),
 ('horrendous', 0.6799140572547913),
 ('horrid', 0.6745357513427734),
 ('appalling', 0.6586296558380127),
 ('amateurish', 0.6267550587654114),
 ('lousy', 0.6240130066871643)]

In [54]:
# Load the model created in Part2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

--- Logging error ---
Traceback (most recent call last):
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "/Users/haewonyum/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
TypeError: not enough arguments for format string
Call stack:
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/haewonyum/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/haewo

In [17]:
type(model.wv.syn0)

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [57]:
## individual word vector is 1*300
# model["flower"]
# check word list in the model
model.wv.index2word

['the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 's',
 'was',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'you',
 't',
 'on',
 'not',
 'he',
 'are',
 'his',
 'have',
 'be',
 'one',
 'all',
 'at',
 'they',
 'by',
 'who',
 'an',
 'from',
 'so',
 'like',
 'there',
 'her',
 'or',
 'just',
 'about',
 'out',
 'has',
 'if',
 'what',
 'some',
 'good',
 'can',
 'more',
 'when',
 'very',
 'she',
 'up',
 'no',
 'time',
 'even',
 'would',
 'my',
 'which',
 'their',
 'story',
 'only',
 'really',
 'see',
 'had',
 'were',
 'well',
 'we',
 'me',
 'than',
 'much',
 'bad',
 'get',
 'been',
 'people',
 'also',
 'into',
 'do',
 'great',
 'other',
 'will',
 'first',
 'because',
 'him',
 'how',
 'most',
 'don',
 'them',
 'made',
 'its',
 'make',
 'then',
 'way',
 'could',
 'too',
 'movies',
 'after',
 'any',
 'characters',
 'character',
 'think',
 'films',
 'two',
 'watch',
 'being',
 'many',
 'plot',
 'seen',
 'never',
 'where',
 'love',
 'life',
 'little',
 'acting

In [25]:
## Averaging feature vectors in a review
def makeFeatureVec(words,model,num_features):
    #Function to average all of the word vectors in a given paragraph
    
    #Pre-initialize an empty numpy array(for speed)
    featureVec = np.zeros((num_features), dtype="float32")
    nwords=0
    
    #Index2word is a list that contains the names of the words in the model's vocabulary.
    #Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)

    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec,model[word])
    
    # divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [59]:
## Numpy Array test
# test = np.zeros(300, dtype="float32")
# test = np.add(test,model['man'])
# test = np.add(test,model['king'])
# test = np.divide(test,2)
# print(test)

In [26]:
def getAvgFeatureVecs(reviews,model,num_feature):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    
    # preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")

    # Loop through reviews
    for review in reviews:
        if(counter%1000==0):
            print("review %d of %d"%(counter,len(reviews)))
        reviewFeatureVecs[counter]=makeFeatureVec(review,model,num_features)
        counter +=1
        
    return reviewFeatureVecs

In [30]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word removal.

print("Creating average feature vecs for train reviews...")
clean_train_reviews=[]
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews,model,num_features)
    
print("Creating average feature vecs for test reviews...")
clean_test_reviews=[]
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews,model,num_features)

Creating average feature vecs for train reviews...


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/haewonyum/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


review 0 of 25000


/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
review 20000 of 25000
review 21000 of 25000
review 22000 of 25000
review 23000 of 25000
review 24000 of 25000
Creating average feature vecs for test reviews...
review 0 of 25000
review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
revi

In [66]:
## Use the average paragraph vectors to rain a random forest
## Note that, as in Part 1, we can only use the labeled training reviews to train the model.

# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)

print("fitting a raondom forest to labeled training data...")
forest = forest.fit(trainDataVecs,train["sentiment"])

fitting a raondom forest to labeled training data...


In [67]:
## Test & Extract results
result = forest.predict(testDataVecs)

## Write the test results
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv",index=False,quoting=3)

### From Words to Paragraphs, Attempt 2: Clustering
K-Means clustering with large K can be very slow; the following code took more than 40 minutes on my computer. Below, we set a timer around the K-Means function to see how long it takes.

In [9]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [10]:
## K-means clustering using sklearn
from sklearn.cluster import KMeans
import time 

In [18]:
start = time.time() # Start time

# Set 'k' to be 1/5th of the vocaulary size, or an average of 10 words per cluster
word_vectors = model.wv.syn0
num_clusters = np.int(word_vectors.shape[0]/10)

# Initialize a k-means object and use it to extract centroids
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end-start
print("Time taken for K-means clustering: %d seconds"%(elapsed))

/Users/haewonyum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


Time taken for K-means clustering: 367 seconds


In [19]:
## Create a Word / Index dictionary, mapping each vocabulary word to a cluster number
word_cluster_map = dict(zip(model.wv.index2word,idx))
# word_cluster_map_values word_cluster_map['word']

In [20]:
word_cluster_map_values = list(word_cluster_map.values())
word_cluster_map_keys = list(word_cluster_map.keys())
word_cluster_map_keys[10]

'that'

In [21]:
## Exploring clusters #1-10
# For the first 10 clusters
for cluster in range(10,20):
    print("\n Cluster %d"%cluster)
    words=[]
    for i in range(0,len(word_cluster_map_values)):
        if(word_cluster_map_values[i]==cluster):
            words.append(word_cluster_map_keys[i])
    print(words) 


 Cluster 10
['collection', 'library', 'domain']

 Cluster 11
['emotional', 'physical', 'honesty', 'warmth', 'conviction', 'complexity', 'sincerity', 'sensitivity', 'sarcasm', 'nuances', 'maturity', 'vulnerability', 'intimacy', 'tenderness', 'resonance', 'familiarity']

 Cluster 12
['jagger', 'muse', 'yossi', 'elena', 'eternally', 'lorraine']

 Cluster 13
['raped', 'frightened', 'terrified', 'paralyzed', 'mutilated', 'poisoned', 'strangled', 'startled', 'implying', 'teased']

 Cluster 14
['experiment', 'catastrophe', 'entity', 'epidemic', 'outbreak', 'occurrence', 'demonstration', 'relic', 'artifact', 'unprecedented', 'aviation', 'eel']

 Cluster 15
['politician', 'mercenary', 'specialist', 'formerly', 'journalism', 'mi', 'constable', 'ceo', 'trader', 'ffolkes', 'gigolo', 'pacifist', 'paco', 'defender', 'climber', 'gunfighter', 'accented', 'satanist', 'southerner', 'diplomat', 'entrepreneur']

 Cluster 16
['outer', 'odyssey', 'apollo', 'trading', 'saturn', 'mutiny', 'nigh']

 Cluster 1

In [22]:
## save clustering model
import pickle
review_kMeans = pickle.dumps(kmeans_clustering)

In [43]:
## now we have a cluster (or "centroid") assignment for each word, and we can define a function 
## to convert reviews into bags-of-centroids. 
## This works just like Bag of Words but uses semantically related clusters instead of individual words:

def review_to_clusters(wordlists,word_cluster_map):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / cluster map
    num_clusters = max(word_cluster_map.values())+1
    
    # Pre-allocate the bag of clusters vector
    bag_of_clusters = np.zeros(num_clusters,dtype="float32")
    
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlists:
        if word in word_cluster_map:
            idx = word_cluster_map[word]
            bag_of_clusters[idx]+=1
    
    return bag_of_clusters

In [55]:
## test of 'review_to_clusters'
review_to_clusters(clean_train_reviews[1], word_cluster_map)[1246]

3.0

In [57]:
## clustering reveiws in train / test

# Transform the training set reviews into bags of centroids
train_reviews_clusters = np.zeros((len(clean_train_reviews),num_clusters),dtype="float32")
counter=0
for review in clean_train_reviews:
    train_reviews_clusters[counter] = review_to_clusters(review, word_cluster_map)
    counter+=1
    
# Transform the test set reviews into bags of centroids
test_reviews_clusters = np.zeros((len(clean_test_reviews),num_clusters),dtype="float32")
counter=0
for review in clean_test_reviews:
    test_reviews_clusters[counter] = review_to_clusters(review, word_cluster_map)
    counter+=1

In [58]:
## Fit a random forest and extract predictions 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_reviews_clusters, train['sentiment'])
result = forest.predict(test_reviews_clusters)

In [75]:
## Write the test results 
output = pd.DataFrame({"id":test['id'],"sentiment":result})
output.to_csv("BagofCentroids.csv", index=False, quoting=3)